In [1]:
## Author Miquéias Fernandes
## Extract alternative sequences from tabular file
## 05/21

import os
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

In [2]:
REMOVE_chr = True

In [3]:
genoma = '../Ca_scaffolds/Ca_scaffolds.fasta'
events = {
    'extract/SE.MATS.JC.txt': [1003],
    'extract/RI.MATS.JC.txt': [667]
}

In [4]:
def importEnventsFromFile(file, events, remove_chr=True):
    res = {}
    file_name = os.path.basename(file)
    is_a3ss = file_name.startswith("A3SS.")
    is_a5ss = file_name.startswith("A5SS.")
    is_ri = file_name.startswith("RI.")
    is_se = file_name.startswith("SE.")
    is_mxe = file_name.startswith("MXE.")

    if not any([is_a3ss, is_a5ss, is_ri, is_se, is_mxe]):
        print('File desconhecido:', file)
        return res

    for _, evt in pd.read_csv(file, delimiter='\t').iterrows():

        GeneID = evt['GeneID'] 
        ID = ('A3SS' if is_a3ss else 
            'A5SS' if is_a5ss else 
            'RI' if is_ri else 
            'SE' if is_se else 
            'MXE') + str(evt['ID']) + '_' + GeneID
        seq = evt['chr'][3:] if remove_chr else evt['chr']
        strand = evt['strand']

        if not evt['ID'] in events:
            continue

        res[evt['ID']] = (ID, GeneID, seq, evt)
    return res

In [5]:
parsed_events = {k: importEnventsFromFile(k, v, remove_chr=REMOVE_chr) for k, v in events.items()}

In [6]:
needed_contigs = set(','.join([','.join((x[2] for x in x.values())) for x in parsed_events.values()]).split(','))

In [7]:
contigs = SeqIO.to_dict((c for c in SeqIO.parse(genoma, 'fasta') if c.id in needed_contigs))

In [8]:
def getAlternativeSequence(events_parsed, fasta, writeOn=None):
    res = []
    for file, events in events_parsed.items():
        for ID, GeneID, seq, evt in events.values():

            is_a3ss = ID.startswith("A3SS")
            is_a5ss = ID.startswith("A5SS")
            is_ri = ID.startswith("RI")
            is_se = ID.startswith("SE")
            is_mxe = ID.startswith("MXE")

            if not any([is_a3ss, is_a5ss, is_ri, is_se, is_mxe]):
                print('File desconhecido:', file)
                continue

            strand = evt['strand']
            
            def toSeq(seqfna, name, strand, a=None, b=None):
                sr = SeqRecord(Seq(seqfna))
                sr = sr if strand == '+' else sr.reverse_complement()
                sr.id=name
                sr.description=f'{seq} {strand}{a}:{b}' if a and b else ''
                return sr

            if is_a3ss or is_a5ss:
                longExonStart_0base = evt['longExonStart_0base']
                longExonEnd = evt['longExonEnd']
                shortES = evt['shortES']
                shortEE = evt['shortEE']
                slong = str(fasta[seq].seq[longExonStart_0base:longExonEnd])
                sshort = str(fasta[seq].seq[shortES:shortEE])
                res.append( toSeq(slong, ID+'_LONG', strand, longExonStart_0base+1, longExonEnd) )
                res.append( toSeq(sshort, ID+'_SHORT', strand, shortES+1, shortEE) )
                res.append( toSeq(slong.replace(sshort, ''), ID+'_ALTERNATIVE', strand) )

            if is_ri:
                riExonStart_0base = evt['riExonStart_0base']
                riExonEnd = evt['riExonEnd']
                sri = str(fasta[seq].seq[riExonStart_0base:riExonEnd])
                res.append( toSeq(sri, ID+'_INTRON_RETIDO', strand, riExonStart_0base+1, riExonEnd) )

            if is_se:
                exonStart_0base = evt['exonStart_0base']
                exonEnd = evt['exonEnd']
                sse = str(fasta[seq].seq[exonStart_0base:exonEnd])
                res.append( toSeq(sse, ID+'_EXON_ALTERNATIVO', strand, exonStart_0base+1, exonEnd) )

            if is_mxe:
                _1stExonStart_0base = evt['1stExonStart_0base']
                _1stExonEnd = evt['1stExonEnd']
                _2ndExonStart_0base = evt['2ndExonStart_0base']
                _2ndExonEnd = evt['2ndExonEnd']
                sse1 = str(fasta[seq].seq[_1stExonStart_0base:_1stExonEnd])
                sse2 = str(fasta[seq].seq[_2ndExonStart_0base:_2ndExonEnd])
                res.append( toSeq(sse1, ID+'_EXON_ALTERNATIVO_1', strand, _1stExonStart_0base+1, _1stExonEnd) )
                res.append( toSeq(sse2, ID+'_EXON_ALTERNATIVO_2', strand, _2ndExonStart_0base+1, _2ndExonEnd) )
                
    if writeOn:
        SeqIO.write(res, writeOn, 'fasta')
    return res

In [9]:
_ = getAlternativeSequence(parsed_events, contigs, 'alternative_sequences.fna')